#GA LA Data Science 7 - Homework 3

For this homework, we will be using the library tweepy to pull data from the Twitter API, 
and then do language detection on it. That is, you will build a classifier to predict what language a given tweet is, based only on its text.

Start by installing tweepy with pip: ```sudo pip install tweepy```

Be sure to consult the documentation at http://tweepy.readthedocs.org/en/v3.3.0/index.html 
read early, read often. **NOTE:** Tweepy is among the most popular Twitter+Python libraries and is recommended by Twitter. However, it lacks complete documentation, e.g. the documentation is missing the Stream class.

For the early parts, you can also look at this blog post: 
http://adilmoujahid.com/posts/2014/07/twitter-analytics/

In order to access Twitter Streaming API, we need to get 4 pieces of information from Twitter: 
    API key, 
    API secret, 
    Access token, and 
    Access token secret. 
Follow the steps below to get all 4 elements:

- Create a twitter account if you do not already have one.
- Go to https://apps.twitter.com/ and log in with your twitter credentials.
- Click "Create New App"
- Fill out the form, agree to the terms, and click "Create your Twitter application"
- In the next page, click on "Keys and Access Tokens" tab, and copy your "API Key" and "API Secret".
- Scroll down and click "Create my access token", and copy your "Access Token" and "Access Token Secret".
- Double-check and ensure you copied the entire tokens rather than just part of them.

Now that that's all set up, let's get get on with the fun stuff!

In [59]:
# To start, let's just get a basic listener set up. All we want to do here is see 
# a stream of data from Twitter. 
# If this works, you should see a bunch of json's in your output console. 
# (If you see anything else -- such as 3-digit numbers or nothing at all -- then it's time to debug.)
import requests
import random
import time
import urllib
import hmac
from hashlib import sha1
import base64
import collections
import pandas as pd
import numpy as np

#Variables that contains the user credentials to access Twitter API 
api_key = b"yGrCEinj1esitqUKnluIVRGFH"
api_secret = b"a4loFJuuob0Ct63O97xxD4vwFqsj5FCTdkEOZsQIYY90FgruPM"


'''
auth_token = api_key + b':' + api_secret
get_b_token = base64.b64encode(auth_token).decode()
p = requests.post('https://api.twitter.com/oauth2/token', headers={'Authorization': 'Basic ' + get_b_token, 'Content-Type': 'application/x-www-form-urlencoded;charset=UTF-8'}, data='grant_type=client_credentials')
'''
bearer_token = 'AAAAAAAAAAAAAAAAAAAAANUxgwAAAAAA03BiCesaCR%2B1WqSd6e6i5oDfI8M%3DV5wVG3HWNovQTSjim1QZRWAHSkV2OYX4cux3n15rltfgYZldUh'


search_url = "https://api.twitter.com/1.1/search/tweets.json"
search_query = {
        "q": "#paris",
        "count": "100",
        "result_type": "mixed"
}

langs = ['au', 'eu', 'en', 'es', 'mx', 'fr', 'zh', 'tw', 'sv', 'su', 'de', 'ro', 'ru', 'sc', 'sa', 'pt', 'uk', 'it']
tags  = ['yolo', 'vive', 'us', 'earth', 'planet', 'math', 'lang', 'hund', 'dog', 'france', 'da', 'peace', 'love', 'family']

df = []

for lang in langs:
    for tag in tags:
        sq = search_query.copy()
        sq['q'] = '#' + tag
        sq['lang'] = lang
        url = search_url + '?' + urllib.parse.urlencode(sq)

        g = requests.get(url, headers={'Authorization': 'Bearer ' + bearer_token})
        statuses = g.json().get('statuses')
        try:
            for status in statuses:
                df.append({'TEXT': status.get('text'), 'LANG': status.get('lang')})
        except:
            print('whoops... not iterable!')
    

d = pd.DataFrame(df)
print(d['LANG'].unique())
d.to_csv('./tweets.csv')

['en' 'es' 'fr' 'zh' 'sv' 'de' 'und' 'ru' 'tl' 'pt' 'uk' 'it']


In [ ]:
# Now that that's working, let's filter our data. 
# This part is actually pretty easy. 
# Change the on_data method of ListenerParser to extract only the 'text' and 'language' fields, if present.
# We also want to be able to retrieve a set number of results, so I have set a max_results parameter 
# in the constructor. Use that in your edit of on_data to make the object only retrieve up to 
# max_results many results. 

# This is a listener that will extract the data we are interested in and print to stdout
# Note this class is the same class as in the last example. Except now, in on_data we
#    are processing the tweets instead of just printing them!
class ListenerParser(StreamListener):
    
    def __init__(self, max_results): 
        super(ListenerParser, self).__init__()
        
        self.texts = []   # List of tweet messages
        self.langs = []   # List of tweet languages
        
        if max_results:
            self.max_results = max_results
        else: 
            self.max_results = float("inf")
    
    def on_data(self, data):
        if len(l.texts) >= self.max_results:
            return False   # stop collecting data
        try:
            # Converts current tweet from JSON to Python data structures
            data = json.loads(data)
            
            # Look inside the 'data' variable, or read the Twitter docs
            text = 'YOUR ANSWER HERE'   # Text of the tweet (from 'data')
            lang = 'YOUR ANSWER HERE'   # Language of the tweet (from 'data')
            
            # Print status every 500 tweets loaded
            if not (len(l.texts) % 500): 
                print(str(len(l.texts)) + ' tweets loaded!')
            
            # Add current text/lang to tweet/language lists
            self.texts.append(text)
            self.langs.append(lang)
        except: 
            pass
        return True

    def on_error(self, status):
        print('ERROR: ', status)

# Now let's get some data! 
# start with 10 results for testing. 
# once testing is done, increase to 10,000
l = ListenerParser(max_results=10) 

auth = OAuthHandler(api_key, api_secret)
auth.set_access_token(access_token, access_token_secret)

stream = Stream(auth, listener = l)
stream.sample()

In [42]:
# Check and make sure that you have the same number of texts as tweets. 
import collections
import pandas as pd
import numpy as np
import pandas as pd
d = pd.read_csv('../my-data/tweets.csv')

lang = []
text = []
for row in d.values:
    if len(row) == 3:
        if row[1] is not np.nan and row[2] is not np.nan:
            lang.append(row[1])
            text.append(row[2])
    


In [31]:
for l in lang:
    if l is np.nan:
        print(l)

I'm going to use words to determine language. I think chars will have a higher rate of collision across mutliple languages.

In [45]:
# Next, some classifiers. 
# Start with logistic regression. 
# print a full classification report after you have trained the classifier and made predictions. 

from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()


from sklearn.cross_validation import train_test_split
x_train, x_test, y_train, y_test = train_test_split(text, lang)

x_train = count_vect.fit_transform(x_train)
x_test = count_vect.transform(x_test)


model = LogisticRegression().fit(x_train, y_train)
preds = model.predict(x_test)

from sklearn import metrics
print(metrics.accuracy_score(y_test, preds))
print(metrics.confusion_matrix(y_test, preds))
print(metrics.classification_report(y_test, preds))

##Not sure why i'm getting: ValueError: x and y must have same first dimension

0.796005099873
[[247   6   5   0   6   8   2   7   0   0   3   0]
 [ 18 322   4   2   8  10   4   0   0   0   2   2]
 [ 13  12 245   6  12  11   3   3   0   0   4   1]
 [  6   4  11 234   5   7   6   1   0   1   4   3]
 [  8   8   9   2 185   9   2   1   0   0   3   2]
 [  8   6  14   3   2 197   2   1   0   0   2   1]
 [  3   2   2   1   8   4 148   3   0   3   3   2]
 [  8   6   2   2   3   2   4 117   0   0   1   4]
 [  0   0   0   0   0   0   0   0   1   0   0   0]
 [  8   0   2   0   0   3  15   2   0  17   1   3]
 [  3  12  11   6   6   4   4   8   0   1  96   5]
 [  9  11   6   2   2   4   2   2   0   0   2  64]]
             precision    recall  f1-score   support

         de       0.75      0.87      0.80       284
         en       0.83      0.87      0.85       372
         es       0.79      0.79      0.79       310
         fr       0.91      0.83      0.87       282
         it       0.78      0.81      0.79       229
         pt       0.76      0.83      0.80       236


In [49]:
# Now do the same with multinomial naive bayes 
from sklearn.naive_bayes import MultinomialNB

print("MultinomialNB:")
nb_model = MultinomialNB().fit(x_train, y_train)
preds = nb_model.predict(x_test)

from sklearn import metrics
print(metrics.accuracy_score(y_test, preds))
print(metrics.confusion_matrix(y_test, preds))
print(metrics.classification_report(y_test, preds))

MultinomialNB:
0.765830854229
[[240  19   2   0   8   8   2   4   0   0   1   0]
 [  6 344   5   0   6   6   2   1   0   0   2   0]
 [  1  30 256   4   7  12   0   0   0   0   0   0]
 [  2  19   9 235   6   6   2   1   0   0   1   1]
 [  7  13  17   2 175  11   1   2   0   0   1   0]
 [  2  17  17   4   2 191   1   0   0   0   1   1]
 [  2  13   3   5   8   5 139   4   0   0   0   0]
 [  7   9   3   3   4  11   1 110   0   0   1   0]
 [  0   1   0   0   0   0   0   0   0   0   0   0]
 [  3  10   1   0   3   7  23   0   0   3   1   0]
 [  2  24  13   9   6  22   6   1   0   0  72   1]
 [  7  32   6   2   1  11   5   0   0   0   3  37]]
             precision    recall  f1-score   support

         de       0.86      0.85      0.85       284
         en       0.65      0.92      0.76       372
         es       0.77      0.83      0.80       310
         fr       0.89      0.83      0.86       282
         it       0.77      0.76      0.77       229
         pt       0.66      0.81      

/root/anaconda3/lib/python3.4/site-packages/sklearn/metrics/metrics.py:1771: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [ ]:
# Wow, that's some very high-dimensional data. 
# To estimate, if we had 150,000 dimensions * 10,000 datapoints * 4 bytes each (assuming we use int32 or float32; would be 8 byes if we are using int64 or float64)
# the dense matrix would take up ~6 GB of memory. 
# 
# your laptop probably doesn't have that. 
# 
# To deal with this dimensionality problem, we have several options. Among them: 
# 1. Reduce dimensionality and convert to a dense array 
# -- use a dimensionality reduction algorithm (but we haven't covered that yet, so that's not really an option)
# -- or set a max_features limit in scikit-learn's CountVectorizer
# 2. Don't use this classifier 
# 
# For now, I'm going to go with 2. 
# My reasoning is: We already have several classifiers that are very good with high-dimensional data. 
# Even if we pared our data down to, say, 5000 dimensions using a max_features limit, 
# we would be throwing away a ton of information, but it would still be pretty tedious 
# to compute. 
# 
# You are free to choose whichever approach you want. 
# 
# But as a freebie, I'm going to choose a different classifier that does support sparse input 
# to use as a third option. This will be a sneak preview of support vector machines.
# We will use sklearn's LinearSVC rather than SVC, because it is much faster to train (linear vs. quadratic or even cubic time)
# and we don't need any of the extra functionality offered by a nonlinear svm that we could get using SVC.

In [50]:
from sklearn.svm import LinearSVC

svm_model = MultinomialNB().fit(x_train, y_train)
preds = svm_model.predict(x_test)

from sklearn import metrics
print(metrics.accuracy_score(y_test, preds))
print(metrics.confusion_matrix(y_test, preds))
print(metrics.classification_report(y_test, preds))

0.765830854229
[[240  19   2   0   8   8   2   4   0   0   1   0]
 [  6 344   5   0   6   6   2   1   0   0   2   0]
 [  1  30 256   4   7  12   0   0   0   0   0   0]
 [  2  19   9 235   6   6   2   1   0   0   1   1]
 [  7  13  17   2 175  11   1   2   0   0   1   0]
 [  2  17  17   4   2 191   1   0   0   0   1   1]
 [  2  13   3   5   8   5 139   4   0   0   0   0]
 [  7   9   3   3   4  11   1 110   0   0   1   0]
 [  0   1   0   0   0   0   0   0   0   0   0   0]
 [  3  10   1   0   3   7  23   0   0   3   1   0]
 [  2  24  13   9   6  22   6   1   0   0  72   1]
 [  7  32   6   2   1  11   5   0   0   0   3  37]]
             precision    recall  f1-score   support

         de       0.86      0.85      0.85       284
         en       0.65      0.92      0.76       372
         es       0.77      0.83      0.80       310
         fr       0.89      0.83      0.86       282
         it       0.77      0.76      0.77       229
         pt       0.66      0.81      0.73       236


/root/anaconda3/lib/python3.4/site-packages/sklearn/metrics/metrics.py:1771: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [ ]:
# With a bit more preprocessing, this score could be substantially higher. 
# 
# Moving on, our data is less-than-ideal in a few ways: 
# 1. Several of the class labels occur only 1 or 2 times; 
# 2. There are only 10,000 datapoints
# 3. We have done almost no preprocessing (of text, or of data after vectorization)
# 
# From here on, this problem set is an open question: How can you improve on these base scores? 
# What's the best you can produce here? 
# 
# Go at it. And record your results in this notebook. 
# 
# Notes: 
# To determine your best score, use a cross validated score with 5 folds. 
# You are still only allowed to use the text. No meta-data! 